**In this notebook will be showed two kind of recomendation systems.
First one, according to the user id and the movie that user'd like to see. It will be recommended or not. For that, It was used suprise library.
Second one, will be evaluated just the title of the movie vs description. So, it is only necessary to write the movie title by using TfidfVectorizer library. According to the similitarities, the algorithm will show the best top5  wich get matched.
Finally, the second way meets with the project requests. Thus, it will be used on my main Script in order to load to FastApi**

**Recomendation System**

In [47]:
#Importing libraries.
import sys 
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import SVD

In [48]:
#Loading data.
df_scores = pd.read_csv('datasets/df_scores.csv')
df_scores.head()

,userId,rating,movieId
0,1,1.0,as680
1,1,4.5,ns2186
2,1,5.0,hs2381
3,1,5.0,ns3663
4,1,5.0,as9500


In [49]:
#verifyibg if all the data is correct.
print(df_scores.shape)
print(df_scores.info())

(11024289, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11024289 entries, 0 to 11024288
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   userId   int64  
 1   rating   float64
 2   movieId  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 252.3+ MB
None


In [50]:
#crating a rating scale based on 1 up to 5.
reader = Reader(rating_scale=(1,5))

In [51]:
#limitating registers to 100000.
registers = 100000
data = Dataset.load_from_df(df_scores[['userId','movieId','rating']][:registers], reader)

In [52]:
# spliting data.
trainset, testset = train_test_split(data, test_size=0.3)

In [53]:
#  the model used will be Singular Value Decomposition.
model = SVD()
# Training the model.
model.fit(trainset)

In [54]:
# Predicting.
predictions = model.test(testset)

In [55]:
#predictions for the top 5.
predictions[:5]

[Prediction(uid=761, iid='hs1926', r_ui=4.5, est=4.23434800011069, details={'was_impossible': False}),
 Prediction(uid=835, iid='as2458', r_ui=4.0, est=3.6494734606921155, details={'was_impossible': False}),
 Prediction(uid=201, iid='ds505', r_ui=5.0, est=3.80077025883166, details={'was_impossible': False}),
 Prediction(uid=400, iid='as7012', r_ui=4.0, est=3.950335302357885, details={'was_impossible': False}),
 Prediction(uid=761, iid='ns1277', r_ui=5.0, est=4.162327654200758, details={'was_impossible': False})]

In [56]:
# user and movie random prediction.
model.predict(134545,'ns5000')

Prediction(uid=134545, iid='ns5000', r_ui=None, est=3.571496451825739, details={'was_impossible': False})

In [57]:
model.predict('ns5000',True)

Prediction(uid='ns5000', iid=True, r_ui=None, est=3.5281714285714285, details={'was_impossible': False})

In [58]:
#loading only id and title column from totalPLatforms file.
df_title = pd.read_csv('datasets/totalPlatforms.csv',usecols=['id','title'])
df_title.head()

,id,title
0,as1,the grand seduction
1,as2,take care good night
2,as3,secrets of deception
3,as4,pink: staying true
4,as5,monster maker


In [59]:
# le's make a recommendation to a user.
user = 45811
rating = 4   #  taking movies wich was rated with 4 or 5 stars.
df_user = df_scores[(df_scores['userId'] == user) & (df_scores['rating'] >= rating)]
df_user = df_user.reset_index(drop=True)
df_user['title'] = df_title['title'].loc[df_user.movieId.index].values
df_user.head(2)

,userId,rating,movieId,title
0,45811,4.0,hs678,the grand seduction
1,45811,4.0,ns3783,take care good night


In [60]:
recommended_user = df_title.iloc[:].copy()
print(recommended_user.shape)
recommended_user.head()

(22998, 2)


,id,title
0,as1,the grand seduction
1,as2,take care good night
2,as3,secrets of deception
3,as4,pink: staying true
4,as5,monster maker


In [61]:
# Extracting movies that user have viewed.
viewed_user = df_scores[df_scores['userId'] == user]
print(viewed_user.shape)
viewed_user.head()

(18276, 3)


,userId,rating,movieId
4439700,45811,4.0,hs678
4439701,45811,3.0,ns5769
4439702,45811,4.0,ns3783
4439703,45811,2.5,ds813
4439704,45811,3.0,hs2294


In [62]:
recommended_user = recommended_user[~recommended_user['id'].isin(viewed_user['movieId'])]
recommended_user = recommended_user.reset_index()
recommended_user = recommended_user.drop(['index'], axis=1)
recommended_user.head(1)

,id,title
0,as1,the grand seduction


In [63]:
recommended_user['Estimate_Score'] = recommended_user['id'].apply(lambda x: model.predict(user, x).est)
recommended_user = recommended_user.sort_values('Estimate_Score', ascending=False)

recommended_user.head()

,id,title,Estimate_Score
4497,ds228,fantastic four: rise of the silver surfer,4.041752
3267,as7211,vaayai moodi pesavum,3.947120
5144,hs179,the simpsons,3.944373
6102,hs2325,he-man and the masters of the universe,3.919899
9231,ns6274,"behind ""the cove"": the quiet japanese speak out",3.908686


Prediction to a user (random selection) and a title (random selection)

In [64]:
def movie_recommended(userId,movieId):
    # Predicting rated gigen by theuser to title selected.
    prediction = model.predict( userId,str(movieId))

    # If the result of the predicttion is more or iqual to 3.5, title is recommended.
    if prediction.est >= 3.5:
        return "Recomended", prediction.est
    else:
        return "Not Recomended", prediction.est

Evaluating function

In [65]:
user_id = df_scores.userId[100]
movie_id = df_scores.movieId[500]
movie_recommended(user_id,movie_id)

('Recomended', 3.67137674735519)

**Now, we're using TfidfVectorizer**

In [66]:
#Importing libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [67]:
# loading data
df = pd.read_csv('datasets/totalPlatforms.csv',
                           usecols=['title','description'])
df.head()

,title,description
0,the grand seduction,a small fishing village must procure a local d...
1,take care good night,a metro family decides to fight a cyber crimin...
2,secrets of deception,after a man discovers his wife is cheating on ...
3,pink: staying true,"pink breaks the mold once again, bringing her ..."
4,monster maker,teenage matt banting wants to work with a famo...


In [68]:
# Getting the description of the movie title given
movie_title = 'toy story'
desc = df[df['title'] == movie_title]['description'].values[0]
desc

'meet woody, buzz and the gang in an adventure full of humor and heart.'

In [69]:
# Computing all the TF-IDF features for all the descriptions
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['description'])
tfidf_matrix

<22998x40306 sparse matrix of type '<class 'numpy.float64'>'
	with 652664 stored elements in Compressed Sparse Row format>

In [70]:
# Computing cosine similarities between movittiel description and the others descriptions
tfidf_similarities = cosine_similarity(tfidf_matrix, tfidf.transform([desc]))
tfidf_similarities

array([[0.01882074],
       [0.0104598 ],
       [0.00587015],
       ...,
       [0.03115434],
       [0.01936165],
       [0.01769325]])

In [71]:
# Getting top 10 index from fims wich have the best match according to TF-IDF
tfidf_similar_movie_indices = tfidf_similarities.argsort(axis=0)[-10:][::-1].flatten()
tfidf_similar_movie_indices

array([11045, 11047, 11046, 11050,   620, 10053, 10279, 21314, 14302,
       22285], dtype=int64)

In [72]:
# Turn indexes into titles showing  off top 5 .
tfidf_similar_movie_titles = df.loc[tfidf_similar_movie_indices, 'title'].tolist()
tfidf_similar_movie_titles[1:6]

['toy story 3',
 'toy story 2',
 'toy story toons: small fry',
 'the hold steady',
 'toy story of terror!']

**As we can see on the list, movie recommended are related to the movie given. In this case was given 'toy story' title y returned toy story 2, 3 and others, wich are films related.**